Solve equation of fluid motion in a pressure driven pipw flow with periodic boundary condition on both end (starts looping when end is reached to start). If started with an intial uniform profile, the Hagen-Poisenuille parabula profile will develop over time.

Momemtum = ∂u/∂t + (u ⋅ ∇) u = − 1/ρ ∇p + ν ∇²u + f

Incompressibility:  ∇ ⋅ u = 0

u:  Velocity (2d vector)

p:  Pressure

f:  Forcing (here =0)

ν:  Kinematic Viscosity

ρ:  Density

t:  Time

∇:  Nabla operator (defining nonlinear convection, 
gradient and divergence)

∇²: Laplace Operator

Scenario:
 infinitely long tube where it is periodic, the leaving fluid, left from the pipe will reenters the entering wall. 
 
 


In [23]:
'''
Scenario

                  constant pressure gradient
                    <<<<<<<<<<------------

                        wall: u=0, v=0
   p    +-----------------------------------------------+   p
   e    |  -->      -->       -->        -->      -->   |   e
   r    |                                               |   r
   i    |  -->      -->       -->        -->      -->   |   i
   o    |                                               |   o
   d    |  -->      -->       -->        -->      -->   |   d
   i    +-----------------------------------------------+   i
   c                    wall: u=0, v=0                      c

-> A rectangular domain (think of a slice from a pipe with
   circular crossection alongside the longitudal axis)
-> The left and right edge are connected by periodicity,
   representing an infinitely long domain in x axis
-> Top and bottom edge represent wall boundary conditions
-> A constant pressure gradient (in x direction) acts on the
   entire domain

After a while, the expected outcome would be a parabola 
flow, due to boundary layer developing by the viscous effects of the fluid.
Thiis wont affect total mass flux in the simulation 
'''

'\nScenario\n\n                  constant pressure gradient\n                    <<<<<<<<<<------------\n\n                        wall: u=0, v=0\n   p    +-----------------------------------------------+   p\n   e    |  -->      -->       -->        -->      -->   |   e\n   r    |                                               |   r\n   i    |  -->      -->       -->        -->      -->   |   i\n   o    |                                               |   o\n   d    |  -->      -->       -->        -->      -->   |   d\n   i    +-----------------------------------------------+   i\n   c                    wall: u=0, v=0                      c\n\n-> A rectangular domain (think of a slice from a pipe with\n   circular crossection alongside the longitudal axis)\n-> The left and right edge are connected by periodicity,\n   representing an infinitely long domain in x axis\n-> Top and bottom edge represent wall boundary conditions\n-> A constant pressure gradient (in x direction) acts on the\


Solution strategy:

We do not have to consider the velocity in y, since it is 0 in this current setup, therefore its derivative is 0 in the current computational domain.

Discretization of the u-momentum equation

    ∂u/∂t + u ∂u/∂x + v ∂u/∂y = - ∂p/∂x + ν ∇²u
                |     |             |       |
                |     ↓             ↓       |
                |    = 0        constant    |
                |                           |
                ↓                           ↓
        central differences         five-point stencil


0. Instantiate the u-solution field with ones except for
   the top and bottom boundary

1. Compute convection by periodic central difference

    u ∂u/∂x ≈ u[i, j] ⋅ (u[i, (j+1)%N] − u[i, (j−1)%N]) / (2 dx)

2. Compute diffusion by periodic five-point stencil

    ν ∇²u ≈ ν (
        + u[i, (j+1)%N]
        + u[(i+1)%N, j]
        + u[i, (j−1)%N]
        + u[(i−1)%N, j]
        − 4 ⋅ u[i, j]
        ) / (dx²)

3. Advance to next step by explicit Euler step

    u ← u + dt ⋅ (− ∂p/∂x + ν ∇²u − u ∂u/∂x)

4. Enfore the wall boundary condition by setting the u velocity
   at the top and bottom boundary to zero

5. Repeat from (1.) until a steady state is reached.


No pressure correction equation has to be solved since the
pressure gradient is prescribed constant throughout the domain.


In [27]:
# import the necessary packages
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
N_POINTS = 11
KINEMATIC_VISCOUSITY = 0.01
TIME_STEP_LENGTH = 0.2
N_TIME_STEPS = 100

PRESSURE_GRADIENT = np.array([-1.0, 0.0])

In [11]:
element_length = 1.0 / (N_POINTS - 1)
x_range = np.linspace(0, 1, N_POINTS)
y_range = np.linspace(0, 1, N_POINTS)
# make a meshgrid to represent the points in the domain
coordinates_x, coordinates_y = np.meshgrid(x_range, y_range)



In [4]:
# Return central difference, but make it periodic using np.roll to % the result, operation is partial u on x
# This is the first step
def central_different_x_periodic(field):
    diff = (
        (
            np.roll(field, shift = 1, axis = 1)
            -
            np.roll(field, shift = -1, axis = 1)
        ) / (2 * element_length)
    )
    return diff

def laplace_periodic(field):
    diff = (
        np.roll(field, shift = 1, axis = 1)
        +
        np.roll(field, shift = -1, axis = 1)
        +
        np.roll(field, shift = 1, axis = 0)
        +
        np.roll(field, shift = -1, axis = 0)
        -
        4 * field
    ) / (element_length ** 2)
    return diff

In [26]:
from matplotlib.animation import PillowWriter

# Animation stuff
writedata = dict(title = "Pressure_Driven_pipe_flow", artist = "me")
writer = PillowWriter(fps = 10, metadata = writedata)

# Initial conditions
velocity_x_prev = np.ones((N_POINTS, N_POINTS))
# the bottom edge
velocity_x_prev[0, :] = 0
velocity_x_prev[-1, :] = 0

# Setup the figure and axis 
fig = plt.figure()

# Do the animation and save it as a gif on the same directory
with writer.saving(fig, "Pressure_Driven_pipe_flow.gif", 100):
    for iter in tqdm(range(N_TIME_STEPS)):
        convection_x = velocity_x_prev * central_different_x_periodic(velocity_x_prev)
        diffusion_x = KINEMATIC_VISCOUSITY * laplace_periodic(velocity_x_prev)  
        
        velocity_x_next = (
            velocity_x_prev
            +
            TIME_STEP_LENGTH
            *(
                -PRESSURE_GRADIENT[0]
                +
                diffusion_x
                -
                convection_x
            )
        )
        
        velocity_x_next[0, :] = 0.0
        velocity_x_next[-1, :]  = 0.0
        
        # Advance in time
        velocity_x_prev = velocity_x_next
         # Rendering
        plt.contourf(coordinates_x, coordinates_y, velocity_x_prev, level = 50)
        #  Show the color of each velocity
        plt.colorbar()
        # Vector plot
        plt.quiver(coordinates_x, coordinates_y, velocity_x_next, np.zeros_like(velocity_x_next))
        
        plt.twiny()
        plt.plot(velocity_x_next[:, 1], coordinates_y[:,1], color = "white")
        plt.draw()
        # Rendering
        contour = plt.contourf(coordinates_x, coordinates_y, velocity_x_prev, levels=50)
        quiver = plt.quiver(coordinates_x, coordinates_y, velocity_x_next, np.zeros_like(velocity_x_next))
        writer.grab_frame()
        plt.clf()

    



  0%|          | 0/100 [00:00<?, ?it/s]/var/folders/bf/xd2nsc5x5qgdz2fsmfpd2dtw0000gn/T/ipykernel_74598/914554675.py:41: UserWarning: The following kwargs were not used by contour: 'level'
  plt.contourf(coordinates_x, coordinates_y, velocity_x_prev, level = 50)
100%|██████████| 100/100 [00:07<00:00, 14.18it/s]


<Figure size 640x480 with 0 Axes>